In [1]:
import os
import time
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import os
from getpass import getpass
from langchain import hub

In [2]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("C:/Users/USER/Desktop/RAG과제/AI챗봇/2024 한권으로 OK 주식과 세금.pdf")

# 페이지 별 문서 로드
docs = loader.load()

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

In [7]:

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

C:\Users\USER\AppData\Local\Temp\ipykernel_31232\895237271.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [8]:
vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

In [9]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [10]:
def load_prompt_from_file(filename): # 파라미터
    try:
        with open(filename, 'r', encoding='utf-8') as file: # with구문은 open으로 연 것을 자동으로 닫아줌
            return file.read() # return 은 반환값
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {filename}")
        return None

prompt_option = input("1, 2, 3 중 하나를 골라주세요").strip()
if prompt_option == '1':
    filename = "C:/Users/USER/Desktop/RAG과제/AI챗봇/Prompts/prompt1.txt"
elif prompt_option == '2':
    filename = "C:/Users/USER/Desktop/RAG과제/AI챗봇/Prompts/prompt2.txt"
elif prompt_option == '3':
    filename = "C:/Users/USER/Desktop/RAG과제/AI챗봇/Prompts/prompt3.txt"
else:
    print("잘못된 번호를 입력하셨습니다.")


file_content = load_prompt_from_file(filename)
print(file_content)



You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 
Context: {context} 
Answer:


In [11]:
# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", file_content),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


In [ ]:
with open("C:/Users/USER/Desktop/RAG과제/AI챗봇/Results/prompt_result.txt", 'a', encoding='utf-8') as f:
	while True:	
		print("========================")
		query = input("질문을 입력하세요: ")
		if query == "quit": # quit를 입력하여 종료
			break
		response = rag_chain_debug.invoke(query)
		print("Final Response:")
		print(response.content)

		f.write("===============\n")
		f.write(f"질문: {query}\n")
		f.write(f"응답: {response.content}\n\n")


Debug Output: 주식시장이 뭐야?
Debug Output: {'context': [Document(metadata={'source': 'C:/Users/USER/Desktop/RAG과제/AI챗봇/2024 한권으로 OK 주식과 세금.pdf', 'page': 39}, page_content='금융투자상품이란 무엇인가요?')], 'question': '주식시장이 뭐야?'}
Final Response:
주식시장은 기업의 주식이 매매되는 시장으로, 투자자들이 주식을 사고팔 수 있는 장소입니다. 주식시장은 기업의 가치를 반영하며, 경제 전반의 건강 상태를 나타내는 지표로도 활용됩니다. 주식시장에서는 거래소를 통해 주식이 거래되며, 이를 통해 자본을 조달하는 기업과 투자자 간의 거래가 이루어집니다.
